# Contents

* Introduction: Description of the purpose of this notebook
* Data Merging: Merging the Pitchfork and Spotify data
* De-dupe URLs: 
* Added variables:

    * Categorical:
        * review_datetime
        * review_year
    * Numeric:
        * reviewer_reviews
        * review_release_difference
        * artist_reviews
        * artist_review_number

# Introduction

This notebook combines two datasets:   
- Album review data scraped from Pitchfork 
- Spotify artist follower data accessed using Spotify's API

Additionally, we transform and add a handful of features to the data:
- review_date_datetime: Datetime transformation of review_date (the date the review was published to Pitchfork)
- artist_reviews: Total number of album reviews by an artist in the dataset
- artist_review_number: Chronological rank of the artists' reviews (e.g. for Taylor Swift's 3rd album in the dataset, this column will say 3)
- reviewer_reviews: Total number of Pitchfork reviews by a reviewer
- review_release_difference: Difference in years between the review date and the album release year (e.g. for an album released in 1993 but reviewed in 2024, this column will say 31)

In [77]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

# Data Merging

##### Pitchfork album review data

In [78]:
df = pd.read_csv('out/pitchfork_reviews.csv')
df

,artist,album,score,genre,label,reviewer,year,review_date,length,url
0,Jeff Tweedy,Twilight Override,8.0,Rock,dBpm,Elizabeth Nelson,2025.0,2025-09-27,1262,https://www.pitchfork.com/reviews/albums/jeff-...
1,Geese,Getting Killed,9.0,Rock,Partisan / Play It Again Sam,Sam Sodomsky,2025.0,2025-09-26,827,https://www.pitchfork.com/reviews/albums/geese...
2,Piotr Kurek,Songs and Bodies,7.8,Experimental,Unsound,Philip Sherburne,2025.0,2025-09-26,602,https://www.pitchfork.com/reviews/albums/piotr...
3,Paul St. Hilaire,w/ the Producers,7.1,Electronic,Kynant / N.E.W.S.,Daniel Bromfield,2025.0,2025-09-26,556,https://www.pitchfork.com/reviews/albums/paul-...
4,Ø,Sysivalo,8.3,Electronic,Sähkö,Philip Sherburne,2025.0,2025-09-25,1170,https://www.pitchfork.com/reviews/albums/o-sys...
...,...,...,...,...,...,...,...,...,...,...
8921,NaN,NaN,9.0,Global,NaN,Michael J. Agovino,NaN,2017-03-07,1111,https://www.pitchfork.com/reviews/albums/22908...
8922,Elliott Smith,Either/Or: Expanded Edition,10.0,Rock,Kill Rock Stars,Matt LeMay,2017.0,2017-03-07,713,https://www.pitchfork.com/reviews/albums/22947...
8923,WHY?,Moh Lhean,7.7,Rap / Rock,Joyful Noise,Ian Cohen,2017.0,2017-03-07,720,https://www.pitchfork.com/reviews/albums/22945...
8924,DJ Seinfeld,Sunrise EP,6.9,Electronic,Endotherm,Jesse Weiss,2017.0,2017-03-06,642,https://www.pitchfork.com/reviews/albums/22937...


##### Spotify artist follower data

In [79]:
artist_id = pd.read_csv('spotify/artist_ids.csv')
artist_id.head()

,artist,artist_id
0,Jeff Tweedy,2rDxtYUzTAYJJE3Bl3Z5IN
1,Geese,0WCo84qtCKfbyIf1lqQWB4
2,Piotr Kurek,0mneo6UHjcOtZBm1Tw8t67
3,Paul St. Hilaire,2z6qOkQVyn7h3XzUuNlRlM
4,Ø,0dgJbQ0bKPyUXco8hEXN7X


In [80]:
artist_followers = pd.read_csv('spotify/artist_followers.csv')
artist_followers.head()

,artist_id,followers_count
0,2rDxtYUzTAYJJE3Bl3Z5IN,106755.0
1,0WCo84qtCKfbyIf1lqQWB4,170530.0
2,0mneo6UHjcOtZBm1Tw8t67,4708.0
3,2z6qOkQVyn7h3XzUuNlRlM,6820.0
4,0dgJbQ0bKPyUXco8hEXN7X,1026730.0


##### Merging Spotify artist_id and artist_follower data into one Spotify dataset

In [81]:
follows = pd.merge(artist_followers, artist_id, on='artist_id')
follows.head()

,artist_id,followers_count,artist
0,2rDxtYUzTAYJJE3Bl3Z5IN,106755.0,Jeff Tweedy
1,0WCo84qtCKfbyIf1lqQWB4,170530.0,Geese
2,0mneo6UHjcOtZBm1Tw8t67,4708.0,Piotr Kurek
3,2z6qOkQVyn7h3XzUuNlRlM,6820.0,Paul St. Hilaire
4,0dgJbQ0bKPyUXco8hEXN7X,1026730.0,Ø


In [82]:
df = pd.merge(df, follows, how='left', on='artist', indicator='matched')

In [83]:
sum(df['matched'] != 'both')

5

There are 5 albums without associated Spotify data:

In [84]:
df.query("matched == 'left_only'")

,artist,album,score,genre,label,reviewer,year,review_date,length,url,artist_id,followers_count,matched
1114,H.R. Giger’s Studiolo,H​.​R. Giger’s Studiolo Vol​.​ 1 & Vol​. ​2,7.6,Electronic / Experimental,Pacific City Sound Visions,Sam Goldner,2024.0,2024-04-30,808,https://www.pitchfork.com/reviews/albums/hr-gi...,NaN,NaN,left_only
3247,Hans-Joachim Roedelius & Tim Story,4 Hands,7.0,Erased Tapes,Erased Tapes,Vanessa Ague,2022.0,2022-02-25,503,https://www.pitchfork.com/reviews/albums/hans-...,NaN,NaN,left_only
4700,Jackson / Baker / Kirshner,So Glossy and So Thin,7.9,Jazz,Astral Spirits,Sadie Sartini Garner,2020.0,2020-09-21,703,https://www.pitchfork.com/reviews/albums/jacks...,NaN,NaN,left_only
8469,Lal & Mike Waterson,Bright Phoebus,8.5,Folk/Country,Domino,Grayson Haver Currin,1972.0,2017-08-11,1068,https://www.pitchfork.com/reviews/albums/lal-a...,NaN,NaN,left_only
8955,Chaz Bundick Meets the Mattson 2,Star Stuff,6.9,Rock,Company,Stuart Berman,2017.0,2017-03-28,733,https://www.pitchfork.com/reviews/albums/23037...,NaN,NaN,left_only


# Get rid of Artist de-dupes

In [85]:
df = df.drop_duplicates(subset='url', keep='first')
df.head()

,artist,album,score,genre,label,reviewer,year,review_date,length,url,artist_id,followers_count,matched
0,Jeff Tweedy,Twilight Override,8.0,Rock,dBpm,Elizabeth Nelson,2025.0,2025-09-27,1262,https://www.pitchfork.com/reviews/albums/jeff-...,2rDxtYUzTAYJJE3Bl3Z5IN,106755.0,both
1,Geese,Getting Killed,9.0,Rock,Partisan / Play It Again Sam,Sam Sodomsky,2025.0,2025-09-26,827,https://www.pitchfork.com/reviews/albums/geese...,0WCo84qtCKfbyIf1lqQWB4,170530.0,both
2,Piotr Kurek,Songs and Bodies,7.8,Experimental,Unsound,Philip Sherburne,2025.0,2025-09-26,602,https://www.pitchfork.com/reviews/albums/piotr...,0mneo6UHjcOtZBm1Tw8t67,4708.0,both
3,Paul St. Hilaire,w/ the Producers,7.1,Electronic,Kynant / N.E.W.S.,Daniel Bromfield,2025.0,2025-09-26,556,https://www.pitchfork.com/reviews/albums/paul-...,2z6qOkQVyn7h3XzUuNlRlM,6820.0,both
4,Ø,Sysivalo,8.3,Electronic,Sähkö,Philip Sherburne,2025.0,2025-09-25,1170,https://www.pitchfork.com/reviews/albums/o-sys...,0dgJbQ0bKPyUXco8hEXN7X,1026730.0,both


# Variable Transformation

### Categorical

##### Year of review - use timestamp to pull the year from review_date

In [87]:
df['review_date_datetime'] = pd.to_datetime(df['review_date'])

In [88]:
df['review_date_year'] = df['review_date_datetime'].dt.year

### Numeric

##### Reviewer prolificness: Added reviewer_reviews, but we should <strong>use a Mixed effects model</strong>

In [89]:
df = pd.merge(df, pd.DataFrame(df.groupby('reviewer').size()).reset_index().rename({0: 'reviewer_reviews'}, axis=1), on='reviewer')

In [90]:
df = df.rename({'review_date_datetime': 'review_datetime', 'year': 'album_year', 'review_date_year': 'review_year'}, axis=1)

##### Difference between Review Date / Year OR Years since album (2025 - 'year')

In [91]:
df['review_release_difference'] = df['review_year'] - df['album_year']

##### Number of albums for each artist AND which album it is for a given artist

##### Artist total reviews

In [92]:
df = pd.merge(df, pd.DataFrame(df.groupby('artist').size()).reset_index().rename({0: 'artist_reviews'}, axis=1),\
          how="left", on='artist')
df = df.sort_values(['artist', 'review_date'])
df['artist_review_number'] = (
    df.groupby('artist').cumcount() + 1
)

In [93]:
df = df.drop('matched', axis=1)

In [94]:
df.head()

,artist,album,score,genre,label,reviewer,album_year,review_date,length,url,artist_id,followers_count,review_datetime,review_year,reviewer_reviews,review_release_difference,artist_reviews,artist_review_number
8633,!!!,Shake the Shudder,7.3,Rock,Warp,Stuart Berman,2017.0,2017-05-24,726,https://www.pitchfork.com/reviews/albums/23257...,2Jc4AEeBTE47KwuKgYOtcL,362467.0,2017-05-24,2017,213,0.0,3.0,1.0
5848,!!!,Wallop,7.2,Rock,Warp,Jesse Dorris,2019.0,2019-09-07,851,https://www.pitchfork.com/reviews/albums/-wallop/,2Jc4AEeBTE47KwuKgYOtcL,362467.0,2019-09-07,2019,61,0.0,3.0,2.0
2959,!!!,Let It Be Blue,7.4,Rock,Warp,Brian Howe,2022.0,2022-05-12,884,https://www.pitchfork.com/reviews/albums/let-i...,2Jc4AEeBTE47KwuKgYOtcL,362467.0,2022-05-12,2022,72,0.0,3.0,3.0
2351,$ilkMoney,"I Don’t Give a Fuck About This Rap Shit, Imma ...",8.4,Rap,DB$B,Dylan Green,2022.0,2022-11-23,957,https://www.pitchfork.com/reviews/albums/silkm...,6hj2p7pzusn5Fcdi35Yedr,72212.0,2022-11-23,2022,149,0.0,2.0,1.0
169,$ilkMoney,WHO WATERS THE WILTING GIVING TREE ONCE THE LE...,8.0,Rap,Lex,Dash Lewis,2025.0,2025-07-18,1009,https://www.pitchfork.com/reviews/albums/silkm...,6hj2p7pzusn5Fcdi35Yedr,72212.0,2025-07-18,2025,53,0.0,2.0,2.0


In [96]:
df.to_csv('merged_data.csv', index=False)